# Preprocess bsr data

The purpose of this notebook is to get the best bsr over a certain time period, as well as the first launch date of a product. We would like to estimate the effect of inital reviews on the long-term bsr.

Steps:
1. Remove the products whose first review date is earlier than the first bsr date. 
2. Calculate min, 10 quantile, 50 quantile bsr over certain time priod.

- For example,for a product whose min date is 1/1/2018. Compute:
  1. min bsr between 1/1/2019 and 12/31/2019 (i.e. 1 full year later, over the following 1 full year period)
  2. min bsr between 1/1/2020 and 12/31/2020 (i.e. 2 full year later, over the following 1 full year period)
  3. min bsr between 1/1/2019 and 3/31/2019 (i.e. 1 full year later, over the follwoing 3 months period)
  4. min bsr between 7/1/2019 and 9/30/2019 (i.e. 1.5 year later, over the following 3 months period)
  5. min bsr between 1/1/2020 and 3/31/2020 (i.e. 2 year later, over the following 3 months period)

3. The first launch date of a product is calculated as 
  - launch_date = min(first_bsr_date, first_review_date)

4. Generate labels based on the selected threshold brs(3000).


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# input folders
in_data = "/content/drive/My Drive/297R-Caps-Pattern/Data/raw"

# intermediate folders
int_data = "/content/drive/My Drive/297R-Caps-Pattern/Data/intermediate"

# output folders
out_data = "/content/drive/My Drive/297R-Caps-Pattern/Data/clean"

In [3]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Load dataset

In [4]:
# load filtered bsr datasets
bsr_full = pd.read_csv(f'{int_data}/bsr_filtered.csv')

In [5]:
# load original review datasets
rev_full = pd.read_csv(f'{in_data}/asin_review_history.csv')
rev_full = rev_full.drop('Unnamed: 0', axis=1)

In [6]:
# drop products without a Bxxxx asin
rev_full = rev_full[rev_full['asin'].str[0] == 'B'].copy()
# drop duplicates based on all columns
rev_full = rev_full.drop_duplicates()

In [7]:
# before drop asin
print(bsr_full.shape)
print(rev_full.shape)

print(bsr_full['asin'].nunique())
print(rev_full['asin'].nunique())

(10418058, 4)
(3818253, 11)
9146
9976


In [8]:
# find product asin with a review before 2017 
rev_asin_before_2017 = rev_full.query('review_date < "2017-01-01"').copy()['asin'].unique()

In [9]:
# num of product that has a review before 2017
rev_asin_before_2017.shape

(5079,)

In [10]:
# remove products from bsr_full with asin in rev_asin_before_2017
bsr = bsr_full.query('asin not in @rev_asin_before_2017').copy()
# remove products from rev_full with asin in rev_asin_before_2017
rev = rev_full.query('asin not in @rev_asin_before_2017').copy()

In [11]:
# after drop asin
print(bsr.shape)
print(rev.shape)

(3530998, 4)
(1458040, 11)


## Process review data


In [12]:
# drop reviewcommentcount since it's all 0
rev = rev.drop('reviewcommentcount', axis=1).copy()
# rename review date to date
rev = rev.rename(columns={'review_date':'date'})

In [13]:
rev.head()

,asin,product_name,review_title,review_text,reviewrating,date,reviewvotes,reviewverifiedpurchase,temp,country_name
6083,B079PWNBZW,"Align DualBiotic, Prebiotic + Probiotic for Me...",Didn't work for me...,The label clearly states that gas or bloating ...,1.0,2018-12-26,224 people found this helpful,True,"Reviewed in the United States on December 26, ...",United States
6084,B079PWNBZW,"Align DualBiotic, Prebiotic + Probiotic for Me...",BLOATED,The flavor is great! I saw another post that m...,5.0,2020-08-03,70 people found this helpful,True,"Reviewed in the United States on August 3, 2020",United States
6085,B079PWNBZW,"Align DualBiotic, Prebiotic + Probiotic for Me...",Actually helps,I always hesitate with probiotics because they...,5.0,2018-09-06,126 people found this helpful,True,"Reviewed in the United States on September 6, ...",United States
6086,B079PWNBZW,"Align DualBiotic, Prebiotic + Probiotic for Me...","Great to lose belly fat, reduce gas, constipat...",Been taking these for sometime now. Have ventu...,5.0,2020-02-18,56 people found this helpful,True,"Reviewed in the United States on February 18, ...",United States
6087,B079PWNBZW,"Align DualBiotic, Prebiotic + Probiotic for Me...",Don’t get the women’s probiotic. Get regular,I usually take the Align probiotic and love it...,1.0,2018-09-15,89 people found this helpful,True,"Reviewed in the United States on September 15,...",United States


In [14]:
# check the min date of rev
rev['date'].min()

'2017-01-01'

# Merge

In [15]:
bsr['date'] = pd.to_datetime(bsr['date'])
rev['date'] = pd.to_datetime(rev['date'])

In [16]:
# product sample is the intersect of reviews and bsrs
rev_prod = rev[['asin']].drop_duplicates().copy()
bsr_prod = bsr[['asin']].drop_duplicates().copy()

prod_sample = rev_prod.merge(bsr_prod, on='asin', how='inner')
print(prod_sample.shape[0], 'product remains')


4146 product remains


In [17]:
rev = rev.merge(prod_sample, on='asin', how='right').copy()
bsr = bsr.merge(prod_sample, on='asin', how='right').copy()

In [ ]:
bsr.to_pickle(f'{out_data}/bsr_after_2017.pickle',index=False)
rev.to_pickle(f'{out_data}/rev_after_2017.pickle',index=False)

# Process BSR

In [18]:
def fill_na_rank(rank_list):
    rank_df = pd.DataFrame(rank_list,columns=['Rank'])
    rank_df = pd.concat([rank_df.ffill(), rank_df.bfill()]).groupby(level=0).mean()['Rank'].values.tolist()
    return rank_df

def get_value(rank):
  return [rank.min(), rank.quantile(0.1), rank.quantile(0.5)]
  
def compute_min_bsr(date, rank):
  date_df = pd.DataFrame(zip(date, rank), columns=['date','rank'])
  date_df['YearMonth'] = pd.to_datetime(date_df['date'].apply(pd.to_datetime).apply(lambda x: x.strftime('%m-%Y')))

  min_month = date_df['YearMonth'].min()
  one_yr_later = min_month + pd.DateOffset(months=12)
  one_half_yr_later = min_month + pd.DateOffset(months=18)
  two_yr_later = min_month + pd.DateOffset(months=24)

  # 1 full year later, for 1 full year
  range_1 = date_df[(date_df['YearMonth'] >= one_yr_later) & (date_df['YearMonth'] < one_yr_later + pd.DateOffset(months=12))]
  # 2 full year later, for 1 full year
  range_2 = date_df[(date_df['YearMonth'] >= two_yr_later) & (date_df['YearMonth'] < two_yr_later + pd.DateOffset(months=12))]
  # 1 full year later, for 3 months
  range_3 = date_df[(date_df['YearMonth'] >= one_yr_later) & (date_df['YearMonth'] < one_yr_later + pd.DateOffset(months=3))]
  # 1.5 full year later, for 3 months
  range_4 = date_df[(date_df['YearMonth'] >= one_half_yr_later) & (date_df['YearMonth'] < one_half_yr_later + pd.DateOffset(months=3))]
  # 2 full year later, for 3 months
  range_5 = date_df[(date_df['YearMonth'] >= two_yr_later) & (date_df['YearMonth'] < two_yr_later + pd.DateOffset(months=3))]
  return [min_month, get_value(range_1['rank']), get_value(range_2['rank']), get_value(range_3['rank']), 
          get_value(range_4['rank']), get_value(range_5['rank'])]


In [19]:
# Sorting by date
bsr_sorted = bsr.sort_values('date')
bsr_sorted.reset_index(inplace=True)

In [20]:
# Grouping by product and creating timelines
grouped_data = bsr_sorted.groupby('asin')
bsr_timelines_by_product = grouped_data['rank'].apply(list).reset_index(name='rank')
bsr_timelines_by_product['date'] = grouped_data['date'].apply(list).reset_index(name='date')['date']

In [21]:
# fill nan value in the rank
bsr_timelines_by_product['filled_rank'] = bsr_timelines_by_product['rank'].apply(fill_na_rank)

for a product whose min date is 1/1/2018.
compute:
1. min bsr between 1/1/2019 and 12/31/2019 (i.e. 1 full year later, for 1 full year)
2. min bsr between 1/1/2020 and 12/31/2020 (i.e. 2 full year later, for 1 full year)
3. min bsr between 1/1/2019 and 3/31/2019 (i.e. 1 full year later, for 3 months)
4. min bsr between 7/1/2019 and 9/30/2019 (i.e. 1.5 year later, for 3 months)
5. min bsr between 1/1/2020 and 3/31/2020 (i.e. 2 year later, for 3 months)

In [22]:
result_all = bsr_timelines_by_product.apply(lambda x: compute_min_bsr(x.date, 
                                                                      x.filled_rank),
                                            axis=1)

In [23]:
min_month = []
after_1_yr_period_12_mo_min_bsr = []
after_1_yr_period_12_mo_10_perc_bsr = []
after_1_yr_period_12_mo_median_bsr = []

after_2_yr_period_12_mo_min_bsr = []
after_2_yr_period_12_mo_10_perc_bsr = []
after_2_yr_period_12_mo_median_bsr = []

after_1_yr_period_3_mo_min_bsr = []
after_1_yr_period_3_mo_10_perc_bsr = []
after_1_yr_period_3_mo_median_bsr = []

after_1_5_yr_period_3_mo_min_bsr = []
after_1_5_yr_period_3_mo_10_perc_bsr = []
after_1_5_yr_period_3_mo_median_bsr = []

after_2_yr_period_3_mo_min_bsr = []
after_2_yr_period_3_mo_10_perc_bsr = []
after_2_yr_period_3_mo_median_bsr = []

for row in result_all:
  min_month.append(row[0])
  after_1_yr_period_12_mo_min_bsr.append(row[1][0])
  after_1_yr_period_12_mo_10_perc_bsr.append(row[1][1])
  after_1_yr_period_12_mo_median_bsr.append(row[1][2])

  after_2_yr_period_12_mo_min_bsr.append(row[2][0])
  after_2_yr_period_12_mo_10_perc_bsr.append(row[2][1])
  after_2_yr_period_12_mo_median_bsr.append(row[2][2])

  after_1_yr_period_3_mo_min_bsr.append(row[3][0])
  after_1_yr_period_3_mo_10_perc_bsr.append(row[3][1])
  after_1_yr_period_3_mo_median_bsr.append(row[3][2])

  after_1_5_yr_period_3_mo_min_bsr.append(row[4][0])
  after_1_5_yr_period_3_mo_10_perc_bsr.append(row[4][1])
  after_1_5_yr_period_3_mo_median_bsr.append(row[4][2])

  after_2_yr_period_3_mo_min_bsr.append(row[5][0])
  after_2_yr_period_3_mo_10_perc_bsr.append(row[5][1])
  after_2_yr_period_3_mo_median_bsr.append(row[5][2])


In [24]:
bsr_timelines_by_product['min_month_bsr'] = min_month

bsr_timelines_by_product['after_1_yr_period_12_mo_min_bsr'] = after_1_yr_period_12_mo_min_bsr
bsr_timelines_by_product['after_1_yr_period_12_mo_10_perc_bsr'] = after_1_yr_period_12_mo_10_perc_bsr
bsr_timelines_by_product['after_1_yr_period_12_mo_median_bsr'] = after_1_yr_period_12_mo_median_bsr

bsr_timelines_by_product['after_2_yr_period_12_mo_min_bsr'] = after_2_yr_period_12_mo_min_bsr
bsr_timelines_by_product['after_2_yr_period_12_mo_10_perc_bsr'] = after_2_yr_period_12_mo_10_perc_bsr
bsr_timelines_by_product['after_2_yr_period_12_mo_median_bsr'] = after_2_yr_period_12_mo_median_bsr

bsr_timelines_by_product['after_1_yr_period_3_mo_min_bsr'] = after_1_yr_period_3_mo_min_bsr
bsr_timelines_by_product['after_1_yr_period_3_mo_10_perc_bsr'] = after_1_yr_period_3_mo_10_perc_bsr
bsr_timelines_by_product['after_1_yr_period_3_mo_median_bsr'] = after_1_yr_period_3_mo_median_bsr

bsr_timelines_by_product['after_1_5_yr_period_3_mo_min_bsr'] = after_1_5_yr_period_3_mo_min_bsr
bsr_timelines_by_product['after_1_5_yr_period_3_mo_10_perc_bsr'] = after_1_5_yr_period_3_mo_10_perc_bsr
bsr_timelines_by_product['after_1_5_yr_period_3_mo_median_bsr'] = after_1_5_yr_period_3_mo_median_bsr

bsr_timelines_by_product['after_2_yr_period_3_mo_min_bsr'] = after_2_yr_period_3_mo_min_bsr
bsr_timelines_by_product['after_2_yr_period_3_mo_10_perc_bsr'] = after_2_yr_period_3_mo_10_perc_bsr
bsr_timelines_by_product['after_2_yr_period_3_mo_median_bsr'] = after_2_yr_period_3_mo_median_bsr

In [25]:
bsr_timelines_by_product.isnull().sum()

asin                                       0
rank                                       0
date                                       0
filled_rank                                0
min_month_bsr                              0
after_1_yr_period_12_mo_min_bsr          455
after_1_yr_period_12_mo_10_perc_bsr      455
after_1_yr_period_12_mo_median_bsr       455
after_2_yr_period_12_mo_min_bsr         1692
after_2_yr_period_12_mo_10_perc_bsr     1692
after_2_yr_period_12_mo_median_bsr      1692
after_1_yr_period_3_mo_min_bsr           455
after_1_yr_period_3_mo_10_perc_bsr       455
after_1_yr_period_3_mo_median_bsr        455
after_1_5_yr_period_3_mo_min_bsr        1082
after_1_5_yr_period_3_mo_10_perc_bsr    1082
after_1_5_yr_period_3_mo_median_bsr     1082
after_2_yr_period_3_mo_min_bsr          1692
after_2_yr_period_3_mo_10_perc_bsr      1692
after_2_yr_period_3_mo_median_bsr       1692
dtype: int64

In [26]:
# before removing
bsr_timelines_by_product.shape

(4146, 20)

In [27]:
# remove products whose whole live period is less than 1 yr
bsr_timelines_by_product = bsr_timelines_by_product[bsr_timelines_by_product['after_1_yr_period_12_mo_min_bsr'].notna()]

In [28]:
# after removing
bsr_timelines_by_product.shape

(3691, 20)

In [29]:
min_bsr_over_time= bsr_timelines_by_product.drop(columns = ['rank','date','filled_rank'])

In [31]:
min_bsr_over_time.to_pickle(f'{int_data}/min_bsr_over_time.pickle')

# Process review

In [32]:
rev['reviewvotes_num'] = rev["reviewvotes"].fillna('0').str.split().str[0].replace('One','1').str.replace(',','').astype('int')
rev = rev.drop(['reviewvotes', 'temp'],axis=1)


In [33]:
# sort by date
rev['date'] = pd.to_datetime(rev['date'])
rev = rev.sort_values(['asin','date']).copy()

# add column year-moth 
rev['year_month'] = rev['date'].dt.strftime('%m-%Y')

# reformat date column 
rev['date'] = rev['date'].dt.strftime('%m-%d-%Y')

# fill nan reviews with empty string
rev['review_text'] = rev['review_text'].fillna('')

assert (pd.isnull(rev['review_text'])).sum() == 0


In [34]:
def make_list(group):
    cols = ['year_month','date', 'product_name', 'review_title', 'review_text', 'reviewvotes_num', 'reviewrating', 'reviewverifiedpurchase', 'country_name']
    listed = {col : group[col].to_list() for col in cols}
    return pd.Series(listed)

listed = rev.groupby(["asin"]).apply(make_list)
listed = listed.reset_index()
listed['product_name'] = [i[0] for i in listed['product_name']]

In [35]:
def get_concat_review(year_month, date, 
                      review_title,review_text,
                           reviewvotes_num,reviewrating,
                           reviewverifiedpurchase,country_name):
  date_df = pd.DataFrame(zip(year_month, date,
                           review_title,review_text,
                           reviewvotes_num,reviewrating,
                           reviewverifiedpurchase,country_name ), columns=cols)
  min_month =  pd.to_datetime(date_df['year_month']).min()
  after_3_mo = min_month + pd.DateOffset(months=3)
  after_6_mo = min_month + pd.DateOffset(months=6)
  # after_12_mo = min_month + pd.DateOffset(months=12)
  date_df['year_month']  =  pd.to_datetime(date_df['year_month'])
  # 0-3 months
  range_3 = date_df[(date_df['year_month'] >= min_month) & (date_df['year_month'] < after_3_mo)]
  # 0-6 months
  range_6 = date_df[(date_df['year_month'] >= min_month) & (date_df['year_month'] < after_6_mo)]
  # # 0-12 months
  # range_12 = date_df[(date_df['year_month'] >= min_month) & (date_df['year_month'] < after_12_mo)]
  
  return {'min_month_rev': min_month,
          '3_mo': make_list(range_3),
          '6_mo': make_list(range_6),
  }
def make_list(range):
  cols = ['review_title', 'review_text', 'reviewvotes_num', 'reviewrating', 'reviewverifiedpurchase', 'country_name']
  listed = {col : range[col].to_list() for col in cols}
  return pd.Series(listed)
cols = ['year_month','date', 'review_title', 
        'review_text', 'reviewvotes_num', 'reviewrating', 'reviewverifiedpurchase', 'country_name']


In [36]:
result_all = listed.apply(lambda x: get_concat_review(x.year_month,
                                                      x.date,
                                                      x.review_title,
                                                      x.review_text,
                                                      x.reviewvotes_num,
                                                      x.reviewrating,
                                                      x.reviewverifiedpurchase,
                                                      x.country_name),axis=1)

In [37]:
results = pd.DataFrame()
for row in result_all:
  df_3_mo = pd.DataFrame([row['3_mo']])
  df_3_mo.columns += '_3_mo'
  df_6_mo = pd.DataFrame([row['6_mo']])
  df_6_mo.columns += '_6_mo'
  df_full = pd.concat([df_3_mo, df_6_mo,],axis=1)
  df_full['min_month_rev'] = row['min_month_rev']
  results = pd.concat([results, df_full], axis=0).reset_index(drop=True)

In [38]:
df_full.columns

Index(['review_title_3_mo', 'review_text_3_mo', 'reviewvotes_num_3_mo',
       'reviewrating_3_mo', 'reviewverifiedpurchase_3_mo', 'country_name_3_mo',
       'review_title_6_mo', 'review_text_6_mo', 'reviewvotes_num_6_mo',
       'reviewrating_6_mo', 'reviewverifiedpurchase_6_mo', 'country_name_6_mo',
       'min_month_rev'],
      dtype='object')

In [39]:
rev_over_time = pd.concat([listed, results], axis=1)

In [40]:
rev_over_time.columns

Index(['asin', 'year_month', 'date', 'product_name', 'review_title',
       'review_text', 'reviewvotes_num', 'reviewrating',
       'reviewverifiedpurchase', 'country_name', 'review_title_3_mo',
       'review_text_3_mo', 'reviewvotes_num_3_mo', 'reviewrating_3_mo',
       'reviewverifiedpurchase_3_mo', 'country_name_3_mo', 'review_title_6_mo',
       'review_text_6_mo', 'reviewvotes_num_6_mo', 'reviewrating_6_mo',
       'reviewverifiedpurchase_6_mo', 'country_name_6_mo', 'min_month_rev'],
      dtype='object')

In [41]:
rev_over_time_short = rev_over_time[['asin','min_month_rev', 'product_name', 'review_title_3_mo', 'review_text_3_mo', 'reviewvotes_num_3_mo',
       'reviewrating_3_mo', 'reviewverifiedpurchase_3_mo', 'country_name_3_mo',
       'review_title_6_mo', 'review_text_6_mo', 'reviewvotes_num_6_mo',
       'reviewrating_6_mo', 'reviewverifiedpurchase_6_mo', 'country_name_6_mo',
       ]]

In [42]:
rev_over_time_short.to_pickle(f'{int_data}/rev_over_time_short.pickle')

In [43]:
merged_data = min_bsr_over_time.merge(rev_over_time_short, on='asin',how='inner')

In [44]:
merged_data['start_month'] = merged_data[['min_month_bsr','min_month_rev']].min(axis=1)

In [46]:
merged_data.to_pickle(f'{out_data}/bsr_rev_classification.pickle')

In [51]:
merged_data.head()

,asin,min_month_bsr,after_1_yr_period_12_mo_min_bsr,after_1_yr_period_12_mo_10_perc_bsr,after_1_yr_period_12_mo_median_bsr,after_2_yr_period_12_mo_min_bsr,after_2_yr_period_12_mo_10_perc_bsr,after_2_yr_period_12_mo_median_bsr,after_1_yr_period_3_mo_min_bsr,after_1_yr_period_3_mo_10_perc_bsr,...,reviewrating_3_mo,reviewverifiedpurchase_3_mo,country_name_3_mo,review_title_6_mo,review_text_6_mo,reviewvotes_num_6_mo,reviewrating_6_mo,reviewverifiedpurchase_6_mo,country_name_6_mo,start_month
0,B00005K9DO,2017-07-01,4507.615385,7284.191071,13995.333333,9757.571429,12561.041667,16430.104167,13085.750000,19051.916667,...,[5.0],[True],[ United States],[Great for pre menopausal women!],"[If used with Evening Primrose oil, DHEA, and ...",[15],[5.0],[True],[ United States],2017-02-01
1,B0009DVYVC,2018-05-01,2022.461538,3749.250000,5272.333333,2733.444444,4973.342857,37476.600000,2022.461538,4528.459524,...,"[5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0]","[False, False, False, False, True, True, True]","[ United States, United States, United State...","[Your kids will love these!, My kids like thes...",[These gummies are great for kids. They are cu...,"[0, 3, 2, 1, 6, 0, 2, 1, 0, 1, 1, 0, 0, 3, 2, ...","[5.0, 4.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, ...","[False, False, False, False, True, True, True,...","[ United States, United States, United State...",2018-05-01
2,B000CL8LAI,2017-07-01,80436.000000,163346.260000,299560.166667,43079.000000,78449.375000,144469.000000,80436.000000,200828.875000,...,[5.0],[True],[ United States],"[Five Stars, Five Stars]","[Great product for speedy recovery., This prod...","[5, 21]","[5.0, 5.0]","[True, True]","[ United States, United States]",2017-07-01
3,B000H8A212,2017-07-01,42585.666667,58437.187879,99417.000000,3011.800000,8904.375000,35505.083333,59629.400000,99415.478571,...,[5.0],[True],[ United States],"[Works within a day!, Don't Buy!, Sundown Echi...",[I have been using Echinacea for many years bu...,"[1, 5, 4]","[5.0, 1.0, 5.0]","[True, True, True]","[ United States, United States, United States]",2017-06-01
4,B000POZG0U,2017-07-01,43471.600000,67546.400000,102598.200000,40251.000000,60004.250000,89162.450000,51325.500000,70571.700000,...,[5.0],[True],[ United States],[so it is nice not to have to buy 100mg tablet...,[This dosage is hard to find. My physician has...,[4],[5.0],[True],[ United States],2017-06-01


In [50]:
print('avg #reviews in 3 months:', np.mean([len(i) for i in merged_data['review_text_3_mo']]))
print('avg #reviews in 6 months:', np.mean([len(i) for i in merged_data['review_text_6_mo']]))

avg #reviews in 3 months: 23.007044161473857
avg #reviews in 6 months: 54.41343809265781


In [53]:
np.mean(merged_data['after_1_yr_period_12_mo_10_perc_bsr']<3000)

0.10647520997019778

In [54]:
np.mean(merged_data['after_1_yr_period_12_mo_min_bsr']<3000)

0.18098076402059063

In [56]:
np.mean(merged_data['after_2_yr_period_12_mo_min_bsr']<3000)

0.12408561365483609